In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [2]:
w, h = 15, 15
base_path = os.path.join('preprocessed_Data', '*.npz')

file_list = glob(base_path)

x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

print(f"x_data length: {len(x_data)}, y_data length: {len(y_data)}")
print(f"First element shape in x_data: {x_data[0].shape}, First element shape in y_data: {y_data[0]}")

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.15, random_state=2024)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|██████████| 10336/10336 [00:46<00:00, 222.76it/s]


x_data length: 5774460, y_data length: 5774460
First element shape in x_data: (15, 15), First element shape in y_data: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
(4908291, 15, 15, 1) (4908291, 225)
(866169, 15, 15, 1) (866169, 225)


In [32]:
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1), kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    layers.Conv2D(128, 7, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    layers.Conv2D(64, 7, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Flatten(),  # Flatten 레이어 추가
    layers.Dense(h * w, activation='sigmoid')  # 타겟 값의 형태에 맞추어 출력
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

c:\Users\gccs4\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 15, 15, 64)     │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 8, 8, 128)      │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 4, 4, 256)      │     1,605,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 2, 2, 256)      │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 2, 2, 128)      │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 2, 2, 64)       │       401,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 2, 2, 1)        │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 225)            │         1,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,230,566 (27.58 MB)

 Trainable params: 7,230,566 (27.58 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)
print(x_train.shape)
print(y_train.shape)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.keras' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),

)

(4258979, 15, 15, 1)
(4258979, 225)
Epoch 1/10
  577/16637 ━━━━━━━━━━━━━━━━━━━━ 2:42:26 607ms/step - acc: 0.0019 - loss: 0.6609

KeyboardInterrupt: 

In [17]:
import tensorflow as tf

# GPU 디바이스 확인
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print("GPU is available")
else:
    print("GPU is not available")

GPU is not available
